# 7506 - Organizacion de Datos - TP N°2

# 1. Librerias

In [5]:
#librerías, no es necesario volverlas a importar
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.utils import compute_class_weight

util = os.path.join('..','0_Data','Util.ipynb')
#Vectorizacion
%run $util

[nltk_data] Downloading package punkt to /home/celeste/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/celeste/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/celeste/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 2. Train-validation-test split

In [6]:
df_train = pd.read_csv(os.path.join('..','dataset','train.csv'),index_col='id', encoding='utf8')
df_test = pd.read_csv(os.path.join('..','dataset','test.csv'),index_col='id', encoding='utf8')

#Le agregamos una columna de texto preprocesado para no tener que hacerlo varias veces
df_train['texto_preprocesado_sentiment_analysis'] = df_train['text'].apply(lambda x: clean_text_sentiment_analysis(x))
df_test['texto_preprocesado_sentiment_analysis'] = df_test['text'].apply(lambda x: clean_text_sentiment_analysis(x))

#Creamos sets de training, validación y test randomizados
#manteneniendo los pesos de cada clase en los splits
temp_df, test_data = train_test_split(df_train, test_size=0.2, random_state=1, shuffle=True, stratify=df_train.target)
train_data, val_data = train_test_split(temp_df, test_size=0.2, random_state=2, shuffle=True, stratify=temp_df.target)                                       
df_train2 = df_train.copy()
df_train2.text = df_train2.texto_preprocesado_sentiment_analysis

df_test2 = df_test.copy()
df_test2.text = df_test2.texto_preprocesado_sentiment_analysis

train_data2 = train_data.copy()
train_data2.text = train_data2.texto_preprocesado_sentiment_analysis

val_data2 = val_data.copy()
val_data2.text = val_data2.texto_preprocesado_sentiment_analysis

test_data2 = test_data.copy()
test_data2.text = test_data2.texto_preprocesado_sentiment_analysis

## Datos originales

In [1]:
class datos_originales_como_arrays:
    def __init__(self):
        self.x_train = train_data.text
        self.y_train = train_data.target
        self.x_val = val_data.text
        self.y_val = val_data.target
        self.x_test = test_data.text
        self.y_test = test_data.target
        self.predict = df_test.text

class datos_originales_como_databases:
    def __init__(self):
        self.train = train_data.copy()
        self.validation = val_data.copy()
        self.test = test_data.copy()
        self.predict = df_test.copy()
        
#Devolver los datos como arrays
def get_data_original_as_np_array():    
    return datos_originales_como_arrays()

#Devolver los datos como databases
def get_data_original_as_database():    
    return datos_originales_como_databases()

#ejemplo: data = get_data_original_as_np_array()
#         model.fit(x=data.x_train, y=data.y_train,... 

## Datos Preprocesados

In [ ]:
class datos_sentiment_analysis_como_np_arrays:
    def __init__(self):        
        self.x_train = train_data.texto_preprocesado_sentiment_analysis
        self.y_train = train_data.target
        self.x_val = val_data.texto_preprocesado_sentiment_analysis
        self.y_val = val_data.target
        self.x_test = test_data.texto_preprocesado_sentiment_analysis
        self.y_test = test_data.target
        self.predict = df_test.texto_preprocesado_sentiment_analysis

class datos_sentiment_analysis_como_np_databases:
    def __init__(self):
        global train_data2
        global val_data2
        global test_data2
        global df_test2
        
        self.train = train_data2
        self.validation = val_data2
        self.test = test_data2
        self.predict = df_test2

#Devolver los datos como arrays
def get_data_sentiment_analysis_as_np_array():    
    return datos_sentiment_analysis_como_np_arrays()

#Devolver los datos como databases
def get_data_sentiment_analysis_as_database():    
    return datos_sentiment_analysis_como_np_databases()


#ejemplo: data = get_data_sentiment_analysis_as_np_array()
#         model.fit(x=data.x_train, y=data.y_train,... 
#         model.evaluate(data.test_x, data.test_y)
# model.predict(predict,...

# 3. K-Fold Cross Validation split

In [ ]:

kf = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)

class datos_originales_k_fold_como_arrays:
    def __init__(self, train_data=None, val_data=None):
        self.x_train = train_data.text
        self.y_train = train_data.target
        self.x_val = val_data.text
        self.y_val = val_data.target

def convertir_a_formato_objeto(train_index, validation_index, train_database):
    entrenamiento = train_database.iloc[train_index]
    validacion = train_database.iloc[validation_index]
    data = datos_originales_k_fold_como_arrays(train_data=entrenamiento, val_data=validacion)    
    return data

## Datos originales

In [ ]:
#Creamos sets de training, validación y test randomizados
#manteneniendo los pesos de cada clase en los splits
    
#Devuelve el split, la base de datos de entrenamiento y la base de datos para predecir
def get_k_folded_data_original_as_np_array():    
    split = kf.split(X = df_train.drop('target', axis=1), y=df_train.target)
    return split, df_train.copy(), df_test.text

def get_k_folded_data_original_as_database():    
    split = kf.split(X = df_train.drop('target', axis=1), y=df_train.target)
    return split, df_train.copy(), df_test.copy()

# ejemplo: 
# split, df_train, predict = get_k_folded_data_original_as_np_array()
# for train_index, validation_index in split:
#         data = convertir_a_formato_objeto(train_index, validation_index, df_train)
#         model.fit(x=data.x_train, y=data.y_train,... 
#         model.evaluate(data.test_x, data.test_y)    
#     ...
# model.predict(predict,...

## Datos Preprocesados

In [ ]:
#Devuelve el split, la base de datos de entrenamiento y la base de datos para predecir
def get_k_folded_data_sentiment_analysis_as_np_array():    
    split = kf.split(X = df_train2.drop('target', axis=1), y=df_train2.target)
    return split, df_train2, df_test2.text

def get_k_folded_data_sentiment_analysis_as_database():    
    split = kf.split(X = df_train2.drop('target', axis=1), y=df_train2.target)
    return split, df_train2, df_test2